In [2]:
import pandas as pd

# read in csv
daily_calories = pd.read_csv("https://raw.githubusercontent.com/qzhang21/BMI706_FinalProject/main/dailyCalories_merged.csv")
daily_steps = pd.read_csv("https://raw.githubusercontent.com/qzhang21/BMI706_FinalProject/main/dailySteps_merged.csv")

print(daily_calories.head())
print(daily_steps.head())




           Id ActivityDay  Calories
0  1503960366   4/12/2016      1985
1  1503960366   4/13/2016      1797
2  1503960366   4/14/2016      1776
3  1503960366   4/15/2016      1745
4  1503960366   4/16/2016      1863
           Id ActivityDay  StepTotal
0  1503960366   4/12/2016      13162
1  1503960366   4/13/2016      10735
2  1503960366   4/14/2016      10460
3  1503960366   4/15/2016       9762
4  1503960366   4/16/2016      12669


In [3]:
daily_steps

,Id,ActivityDay,StepTotal
0,1503960366,4/12/2016,13162
1,1503960366,4/13/2016,10735
2,1503960366,4/14/2016,10460
3,1503960366,4/15/2016,9762
4,1503960366,4/16/2016,12669
...,...,...,...
935,8877689391,5/8/2016,10686
936,8877689391,5/9/2016,20226
937,8877689391,5/10/2016,10733
938,8877689391,5/11/2016,21420


In [4]:
import numpy as np
np.unique(daily_calories['Id'])

# when merging, the order MUST be activity_df.merge(category_df) (i.e. category column is the last column of the merged_df)
test_df = daily_calories.merge(daily_steps, on=["Id", "ActivityDay"]) # merge files
test_df.head()

,Id,ActivityDay,Calories,StepTotal
0,1503960366,4/12/2016,1985,13162
1,1503960366,4/13/2016,1797,10735
2,1503960366,4/14/2016,1776,10460
3,1503960366,4/15/2016,1745,9762
4,1503960366,4/16/2016,1863,12669


In [5]:
import streamlit as st
import altair as alt

chart = alt.Chart(test_df).mark_circle().encode(
    x=alt.X("StepTotal"),
    y=alt.Y('Calories'),
    color=alt.Color("Id", type="nominal"),
    tooltip=["StepTotal", "Calories" ],
).properties(
    title="hi",
)

chart

alt.Chart(...)

In [6]:
quantile_df = test_df.quantile(q=[.25, 0.50, 0.75], axis = 0)
q1 = float(quantile_df.iloc[0, [-1]])
q2 = float(quantile_df.iloc[1, [-1]])
q3 = float(quantile_df.iloc[2, [-1]])


In [7]:
# need to take this quantile, see what everyone fall into, and assign them to the quantile column.
# second plot, also plot Q1,2,3,4. This is to show how many days do individuals are within the quantiles
index_q1 = np.where(test_df.iloc[:, [-1]] < q1)[0] # gets the index of the df matching the condition. [0] to get the index
index_q2 = np.where((test_df.iloc[:, [-1]] >= q1) & (test_df.iloc[:, [-1]] < q2))[0]
index_q3 = np.where((test_df.iloc[:, [-1]] >= q2) & (test_df.iloc[:, [-1]] < q3))[0]
index_q4 = np.where(test_df.iloc[:, [-1]] >= q3)[0]




In [8]:
# assign quantiles
test_df['Quantile'] = None
test_df.loc[index_q1, 'Quantile'] = "Q1"
test_df.loc[index_q2, 'Quantile'] = "Q2"
test_df.loc[index_q3, 'Quantile'] = "Q3"
test_df.loc[index_q4, 'Quantile'] = "Q4"
test_df.head()

,Id,ActivityDay,Calories,StepTotal,Quantile
0,1503960366,4/12/2016,1985,13162,Q4
1,1503960366,4/13/2016,1797,10735,Q4
2,1503960366,4/14/2016,1776,10460,Q3
3,1503960366,4/15/2016,1745,9762,Q3
4,1503960366,4/16/2016,1863,12669,Q4


In [22]:
# test_df.head(50)

In [ ]:
alt.Chart(test_df).transform_density(
    'Miles_per_Gallon',
    as_=['Miles_per_Gallon', 'density'],
    extent=[5, 50],
    groupby=['Origin']
).mark_area(orient='horizontal').encode(
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'Origin:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
        ),
    )
)

In [35]:
# selection = alt.selection_single(fields=['Quantile'], bind='legend')

alt.Chart(test_df).mark_boxplot(extent='min-max').encode(
    x='Quantile:O',
    y='Calories:Q',
    color='Quantile',
    # opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=400
)

alt.Chart(...)

In [25]:
alt.selection_single(fields=['Quantile'], bind='legend')

Selection('selector007', SelectionDef({
  bind: 'legend',
  fields: ['Quantile'],
  type: 'single'
}))

In [ ]:
# wyc try

In [23]:
df_c = daily_calories.copy()
df_s = daily_steps.copy()
df_c["ActivityDay"] = pd.to_datetime(df_c["ActivityDay"])
df_s["ActivityDay"] = pd.to_datetime(df_s["ActivityDay"])
df_c.head()

,Id,ActivityDay,Calories
0,1503960366,2016-04-12,1985
1,1503960366,2016-04-13,1797
2,1503960366,2016-04-14,1776
3,1503960366,2016-04-15,1745
4,1503960366,2016-04-16,1863


In [44]:
start_dates = {}
for id, frame in df_c.sort_values(by="ActivityDay").groupby("Id"):
    start_dates[id] = frame.iloc[0]["ActivityDay"]

In [52]:
df_c["lapse"] = [(row["ActivityDay"] - start_dates[row["Id"]]).days  for i, row in df_c.iterrows()]
df_c.head()

,Id,ActivityDay,Calories,lapse
0,1503960366,2016-04-12,1985,0
1,1503960366,2016-04-13,1797,1
2,1503960366,2016-04-14,1776,2
3,1503960366,2016-04-15,1745,3
4,1503960366,2016-04-16,1863,4


In [53]:

for id, frame in df_c.sort_values(by="ActivityDay").groupby("Id"):
    print(f"For {id}")
    print("------------------------")
    print(frame.head(5), end="\n\n")

For 1503960366
------------------------
           Id ActivityDay  Calories  lapse
0  1503960366  2016-04-12      1985      0
1  1503960366  2016-04-13      1797      1
2  1503960366  2016-04-14      1776      2
3  1503960366  2016-04-15      1745      3
4  1503960366  2016-04-16      1863      4

For 1624580081
------------------------
            Id ActivityDay  Calories  lapse
31  1624580081  2016-04-12      1432      0
32  1624580081  2016-04-13      1411      1
33  1624580081  2016-04-14      1572      2
34  1624580081  2016-04-15      1344      3
35  1624580081  2016-04-16      1463      4

For 1644430081
------------------------
            Id ActivityDay  Calories  lapse
62  1644430081  2016-04-12      3199      0
63  1644430081  2016-04-13      2902      1
64  1644430081  2016-04-14      3226      2
65  1644430081  2016-04-15      2750      3
66  1644430081  2016-04-16      3493      4

For 1844505072
------------------------
            Id ActivityDay  Calories  lapse
92  184

In [55]:

chart = alt.Chart(df_c).mark_line().encode(
    x=alt.X("lapse"),
    y=alt.Y("Calories"),
    color=alt.Color("Id", type="nominal"),
    tooltip=["lapse", "Calories" ],
).properties(
    title="hi",
)

chart

alt.Chart(...)